Notebook to convert data into panel format for regressions

In [1]:
import pandas as pd

In [2]:
# Load aggregated dataframes
month_agg = pd.read_csv('data/aggregated_data.csv')
week_agg = pd.read_csv('data/aggregated_data_weekly.csv')

In [3]:
month_agg.head()

,Unnamed: 0,year_month,launched,share_with_ukraine,a3_share_with_ukraine,la6_share_with_ukraine,t5_share_with_ukraine,fatalities,a3_war_share_out_of_total,la6_war_share_out_of_total,t5_war_share_out_of_total,combined_war_share_out_of_total,a3_share_class_1,la6_share_class_1,t5_share_class_1,total_share_class_1
0,0,2022-12-01,364.0,0.068872,0.091422,0.059908,0.058140,2174,0.019943,0.013344,0.011649,0.014071,0.172467,0.212945,0.198678,0.196723
1,1,2023-01-01,213.0,0.102060,0.147257,0.079872,0.079861,1187,0.022056,0.025839,0.025656,0.025016,0.255780,0.321944,0.318334,0.306667
2,2,2023-02-01,193.0,0.141184,0.195722,0.100925,0.137572,665,0.037432,0.021755,0.028227,0.025540,0.220494,0.215436,0.204096,0.211006
3,3,2023-03-01,181.0,0.090881,0.111977,0.076399,0.088000,866,0.025806,0.019705,0.025962,0.022247,0.265193,0.256988,0.292427,0.272245
4,4,2023-04-01,117.0,0.072224,0.091097,0.061329,0.065614,1135,0.022167,0.013754,0.018753,0.016805,0.221421,0.236270,0.283354,0.250800


In [4]:
week_agg.head()

,Unnamed: 0,week,launched,share_with_ukraine,a3_share_with_ukraine,la6_share_with_ukraine,t5_share_with_ukraine,fatalities,a3_war_share_out_of_total,la6_war_share_out_of_total,t5_war_share_out_of_total,count_class_one,total_seconds,combined_war_share_out_of_total,a3_share_class_1,la6_share_class_1,t5_share_class_1,total_share_class_1
0,0,2022-12-05,99.0,0.082557,0.082251,0.081761,0.084158,313,0.017424,0.018501,0.013284,583.0,35160,0.016581,0.143154,0.225271,0.156615,0.189470
1,1,2022-12-12,116.0,0.038674,0.048458,0.046429,0.018433,282,0.006019,0.007953,0.004147,182.0,29640,0.006140,0.213115,0.173454,0.222222,0.188211
2,2,2022-12-19,38.0,0.088710,0.115000,0.072581,0.081395,646,0.016417,0.017675,0.015213,617.0,37200,0.016586,0.142444,0.242173,0.185579,0.186124
3,3,2022-12-26,156.0,0.077961,0.131250,0.051351,0.087591,972,0.028061,0.011468,0.028102,610.0,32760,0.018620,0.183130,0.196872,0.318621,0.224843
4,4,2023-01-02,41.0,0.076590,0.074257,0.088123,0.065502,211,0.006173,0.021201,0.013173,563.0,37500,0.015013,0.103950,0.239711,0.198901,0.202810


In [5]:
# Drop cols which are aggregated across channel
month_agg = month_agg.drop(['share_with_ukraine', 'total_share_class_1', 'combined_war_share_out_of_total'], axis=1)
week_agg = week_agg.drop(['share_with_ukraine', 'total_share_class_1', 'combined_war_share_out_of_total'], axis=1)

Convert month df to panel

In [10]:
# Melting the dataframe to long format
df_long = pd.melt(month_agg, id_vars=['year_month', 'launched', 'fatalities'],
                  value_vars=['a3_share_with_ukraine', 'la6_share_with_ukraine', 't5_share_with_ukraine', 
                              'a3_share_class_1', 'la6_share_class_1', 't5_share_class_1',
                              'a3_war_share_out_of_total', 'la6_war_share_out_of_total', 't5_war_share_out_of_total'],
                  var_name='type', value_name='value')

# Extracting the 'a3', 't5', 'la6' part and the share type part
df_long[['share_type', 'type']] = df_long['type'].str.extract(r'(a3|la6|t5)_(.*)')

# Pivoting the dataframe to wide format again
df_panel = df_long.pivot_table(index=['year_month', 'launched', 'fatalities', 'share_type'], 
                               columns='type', values='value').reset_index()

# Reordering the columns
df_panel_month = df_panel[['year_month', 'share_type', 'launched', 
                     'share_with_ukraine', 'fatalities', 'share_class_1', 'war_share_out_of_total']]

# Renaming columns
df_panel_month.columns = ['year_month', 'channel', 'launched', 
                    'coverage', 'fatalities', 'war_images', 'war_share_out_of_total']

In [11]:
df_panel_month

,year_month,channel,launched,coverage,fatalities,war_images,war_share_out_of_total
0,2022-12-01,a3,364.0,0.091422,2174,0.172467,0.019943
1,2022-12-01,la6,364.0,0.059908,2174,0.212945,0.013344
2,2022-12-01,t5,364.0,0.058140,2174,0.198678,0.011649
3,2023-01-01,a3,213.0,0.147257,1187,0.255780,0.022056
4,2023-01-01,la6,213.0,0.079872,1187,0.321944,0.025839
5,2023-01-01,t5,213.0,0.079861,1187,0.318334,0.025656
6,2023-02-01,a3,193.0,0.195722,665,0.220494,0.037432
7,2023-02-01,la6,193.0,0.100925,665,0.215436,0.021755
8,2023-02-01,t5,193.0,0.137572,665,0.204096,0.028227
9,2023-03-01,a3,181.0,0.111977,866,0.265193,0.025806


Convert week df to panel

In [12]:
# Melting the dataframe to long format
df_long = pd.melt(week_agg, id_vars=['week', 'launched', 'fatalities'],
                  value_vars=['a3_share_with_ukraine', 'la6_share_with_ukraine', 't5_share_with_ukraine', 
                              'a3_share_class_1', 'la6_share_class_1', 't5_share_class_1',
                              'a3_war_share_out_of_total', 'la6_war_share_out_of_total', 't5_war_share_out_of_total'],
                  var_name='type', value_name='value')

# Extracting the 'a3', 't5', 'la6' part and the share type part
df_long[['share_type', 'type']] = df_long['type'].str.extract(r'(a3|la6|t5)_(.*)')

# Pivoting the dataframe to wide format again
df_panel = df_long.pivot_table(index=['week', 'launched', 'fatalities', 'share_type'], 
                               columns='type', values='value').reset_index()

# Reordering the columns
df_panel_week = df_panel[['week', 'share_type', 'launched', 
                     'share_with_ukraine', 'fatalities', 'share_class_1', 'war_share_out_of_total']]

# Renaming columns
df_panel_week.columns = ['week', 'channel', 'launched', 
                    'coverage', 'fatalities', 'war_images', 'war_share_out_of_total']

In [13]:
df_panel_week

,week,channel,launched,coverage,fatalities,war_images,war_share_out_of_total
0,2022-12-05,a3,99.0,0.082251,313,0.143154,0.017424
1,2022-12-05,la6,99.0,0.081761,313,0.225271,0.018501
2,2022-12-05,t5,99.0,0.084158,313,0.156615,0.013284
3,2022-12-12,a3,116.0,0.048458,282,0.213115,0.006019
4,2022-12-12,la6,116.0,0.046429,282,0.173454,0.007953
...,...,...,...,...,...,...,...
210,2024-04-22,a3,92.0,0.030973,135,0.228029,0.007080
211,2024-04-22,la6,92.0,0.056818,135,0.327815,0.018750
212,2024-04-22,t5,92.0,0.000000,135,0.338798,0.005616
213,2024-04-29,a3,56.0,0.032258,67,0.250518,0.006452


Save dataframes

In [14]:
df_panel_month.to_csv('data/monthly_panel.csv', index=False)
df_panel_week.to_csv('data/weekly_panel.csv', index=False)